# **Introduction to Google Colab**

## **Jupyter notebook basics**

### **Code cells**

*Interactively use the code blocks*

In [ ]:
2*5

10

In [ ]:
import pandas as pd

## **Access to the shell**

*You can use unix shell commands too using !*

In [ ]:
!ls

sample_data


In [ ]:
!pwd

/content


## **Install Spark**

In [ ]:
# setting up spark environment (This section downloads the files needed)
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [51]:
# This section sets up the environment 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [52]:
!ls

reported_crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [49]:
# import spark and setting up
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [50]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

# **Download Chicago's Reported Crime Data**

In [ ]:
# get the file using shell
# going to take time as its a huge file of about 1-2 GB
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2021-03-29 10:21:36--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [  <=>               ]   1.60G  2.98MB/s    in 8m 51s  

2021-03-29 10:30:27 (3.09 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1721959109]



In [ ]:
# let's list the data and check
!ls

'rows.csv?accessType=DOWNLOAD'	 spark-2.3.1-bin-hadoop2.7
 sample_data			 spark-2.3.1-bin-hadoop2.7.tgz


In [ ]:
# now we can change the name of our datafile to some meaningful
#use tab to complete the file pattern

!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv

In [ ]:
!ls

reported_crimes.csv  spark-2.3.1-bin-hadoop2.7
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [ ]:
# let's check if the data is downloaded or not

from pyspark.sql.functions import to_timestamp,col,lit

# here we also letting spark infer the schema
rc = spark.read.csv('reported_crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5,False)

# Above we are reading the data file as csv , adding a date column with provided MM/dd/yyyy hh:mm:ss a(am/pm) format
# Then filtering only the records before 2018-11-11


+--------+-----------+-------------------+---------------------+----+------------+-----------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+
|ID      |Case Number|Date               |Block                |IUCR|Primary Type|Description            |Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On            |Latitude    |Longitude    |Location                     |
+--------+-----------+-------------------+---------------------+----+------------+-----------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+
|10224738|HY411648   |2015-09-05 13:30:00|043XX S WOOD ST      |0486|BATTERY     |DOMESTIC BATTERY SIMPLE|

## **Working with Dataframes**

**Reading a CSV file in Pyspark**

*df=spark.read.csv(filepath,header=)*

**To get the data or view it in Pyspark**

*Pyspark - df.take(n),df.collect(),df.show(n),df.head(n),df.limit(n)*

In [ ]:
rc.take(1)

[Row(ID='10224738', Case Number='HY411648', Date=datetime.datetime(2015, 9, 5, 13, 30), Block='043XX S WOOD ST', IUCR='0486', Primary Type='BATTERY', Description='DOMESTIC BATTERY SIMPLE', Location Description='RESIDENCE', Arrest='false', Domestic='true', Beat='0924', District='009', Ward='12', Community Area='61', FBI Code='08B', X Coordinate='1165074', Y Coordinate='1875917', Year='2015', Updated On='02/10/2018 03:50:01 PM', Latitude='41.815117282', Longitude='-87.669999562', Location='(41.815117282, -87.669999562)')]

In [ ]:
rc.head()

Row(ID='10224738', Case Number='HY411648', Date=datetime.datetime(2015, 9, 5, 13, 30), Block='043XX S WOOD ST', IUCR='0486', Primary Type='BATTERY', Description='DOMESTIC BATTERY SIMPLE', Location Description='RESIDENCE', Arrest='false', Domestic='true', Beat='0924', District='009', Ward='12', Community Area='61', FBI Code='08B', X Coordinate='1165074', Y Coordinate='1875917', Year='2015', Updated On='02/10/2018 03:50:01 PM', Latitude='41.815117282', Longitude='-87.669999562', Location='(41.815117282, -87.669999562)')

In [ ]:
rc.first()

Row(ID='10224738', Case Number='HY411648', Date=datetime.datetime(2015, 9, 5, 13, 30), Block='043XX S WOOD ST', IUCR='0486', Primary Type='BATTERY', Description='DOMESTIC BATTERY SIMPLE', Location Description='RESIDENCE', Arrest='false', Domestic='true', Beat='0924', District='009', Ward='12', Community Area='61', FBI Code='08B', X Coordinate='1165074', Y Coordinate='1875917', Year='2015', Updated On='02/10/2018 03:50:01 PM', Latitude='41.815117282', Longitude='-87.669999562', Location='(41.815117282, -87.669999562)')

In [ ]:
rc.show(3)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [ ]:
rc.limit(1)

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

In [ ]:
rc.limit(3).show()

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [ ]:
# Taking another example to see difference between limit,take,collect,first,head

simpleData = [("James",34),("Ann",34),("Michael",33),("Scott",53),("Robert",37),("Chad",27)]
columns = ["firstname","age"]
df = spark.createDataFrame(data= simpleData, schema= columns)
df.show()

+---------+---+
|firstname|age|
+---------+---+
|    James| 34|
|      Ann| 34|
|  Michael| 33|
|    Scott| 53|
|   Robert| 37|
|     Chad| 27|
+---------+---+



In [ ]:
print(df.take(3))

[Row(firstname='James', age=34), Row(firstname='Ann', age=34), Row(firstname='Michael', age=33)]


In [ ]:
print(df.first())

Row(firstname='James', age=34)


In [ ]:
print(df.head())

Row(firstname='James', age=34)


In [ ]:
print(df.collect())

[Row(firstname='James', age=34), Row(firstname='Ann', age=34), Row(firstname='Michael', age=33), Row(firstname='Scott', age=53), Row(firstname='Robert', age=37), Row(firstname='Chad', age=27)]


In [ ]:
df.limit(3).show()

+---------+---+
|firstname|age|
+---------+---+
|    James| 34|
|      Ann| 34|
|  Michael| 33|
+---------+---+



## **Schemas : PySpark**

*df.dtypes(),df.printSchema()*

In [40]:
!ls

reported_crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [41]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [42]:
#let's import different datatypes from pyspark
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

In [43]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [57]:
# Now lets explicitly define the schema
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

schema = StructType([
                     StructField('ID',StringType(),True),
                     StructField('Case Number',StringType(),True),
                     StructField('Date',TimestampType(),True),
                     StructField('Block',StringType(),True),
                     StructField('IUCR',StringType(),True),
                     StructField('Primary Type',StringType(),True),
                     StructField('Description',StringType(),True),
                     StructField('Location Description',StringType(),True),
                     StructField('Arrest',StringType(),True),
                     StructField('Domestic',BooleanType(),True),
                     StructField('Beat',StringType(),True),
                     StructField('District',StringType(),True),
                     StructField('Ward',StringType(),True),
                     StructField('Community Area',StringType(),True),
                     StructField('FBI Code',StringType(),True),
                     StructField('X Coordinate',StringType(),True),
                     StructField('Y Coordinate',StringType(),True),
                     StructField('Year',IntegerType(),True),
                     StructField('Updated On',StringType(),True),
                     StructField('Latitude',DoubleType(),True),
                     StructField('Longitude',DoubleType(),True),
                     StructField('Location',StringType(),True)])

In [58]:
# we can also try a shortcut where we can create list of tuple where first element will be column name and second one will be datatype
labels = [('ID',StringType()),
('Case Number',StringType()),
('Date',TimestampType()),
('Block',StringType()),
('IUCR',StringType()),
('Primary Type',StringType()),
('Description',StringType()),
('Location Description',StringType()),
('Arrest',StringType()),
('Domestic',BooleanType()),
('Beat',StringType()),
('District',StringType()),
('Ward',StringType()),
('Community Area',StringType()),
('FBI Code',StringType()),
('X Coordinate',StringType()),
('Y Coordinate',StringType()),
('Year',IntegerType()),
('Updated On',StringType()),
('Latitude',DoubleType()),
('Longitude',DoubleType()),
('Location',StringType())]

In [59]:
# then we can use list comprehension to automatically populate the struct field and struct Type giving us the schema

schema_1 = StructType([StructField (x[0],x[1],True) for x in labels])
schema_1

StructType(List(StructField(ID,StringType,true),StructField(Case Number,StringType,true),StructField(Date,TimestampType,true),StructField(Block,StringType,true),StructField(IUCR,StringType,true),StructField(Primary Type,StringType,true),StructField(Description,StringType,true),StructField(Location Description,StringType,true),StructField(Arrest,StringType,true),StructField(Domestic,BooleanType,true),StructField(Beat,StringType,true),StructField(District,StringType,true),StructField(Ward,StringType,true),StructField(Community Area,StringType,true),StructField(FBI Code,StringType,true),StructField(X Coordinate,StringType,true),StructField(Y Coordinate,StringType,true),StructField(Year,IntegerType,true),StructField(Updated On,StringType,true),StructField(Latitude,DoubleType,true),StructField(Longitude,DoubleType,true),StructField(Location,StringType,true)))

In [60]:
# Not letting spark infer the schema
rc1 = spark.read.csv('reported_crimes.csv',schema=schema_1)
rc1.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [61]:
# lets check the first rows of our dataframe 

rc1.show(5,False)

#So we see issue as few columns are showing nulls
#meaning some entries dnt confer with the datatypes provided,like some cannot be converted to integers or date types

+----+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|ID  |Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+----+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|null|null       |null|null |null|null        |null       |null                |null  |null    |null|null    |null|null          |null    |null        |null        |null|null      |null    |null     |null    |
|null|null       |null|null |null|null        |null       |null                |null  |null    |null|null    |null|null          |null    |null        |null    

## **Working with Columns**

In [62]:
# Select the columns -- To display only first 5 rows of the IUCR column in DF

rc.select(col('IUCR')).show(5)



+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [63]:
# Also you can use
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [64]:
# one more method
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [65]:
# Now lets display first 4 rows of column names Case Number, Date and Arrest

rc.select('Case Number','Date','Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   HY411648|2015-09-05 13:30:00| false|
|   HY411615|2015-09-04 11:30:00| false|
|   JC213529|2018-09-01 00:01:00| false|
|   HY411595|2015-09-05 12:45:00|  true|
+-----------+-------------------+------+
only showing top 4 rows



In [68]:
# Now add a column with name One with entries all 1s

rc.withColumn('One',lit(1)).show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|

In [69]:
# Now let's Remove a column IUCR

rc = rc.drop('IUCR')
rc.show(5)

+--------+-----------+-------------------+--------------------+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     009|  12|            6

## **Working with Rows**

In [70]:
!ls

reported_crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [71]:
# Add the reported crimes for an additional day, 12-Nov-2018 to our dataset

# create dataframe for one day (i.e filter 2018-11-12)

one_day = spark.read.csv('reported_crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))

In [72]:
one_day.count()

3

In [73]:
one_day.show()

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false| 

In [75]:
rc2 = spark.read.csv('reported_crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc2.show(5,False)

+--------+-----------+-------------------+---------------------+----+------------+-----------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+
|ID      |Case Number|Date               |Block                |IUCR|Primary Type|Description            |Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On            |Latitude    |Longitude    |Location                     |
+--------+-----------+-------------------+---------------------+----+------------+-----------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+
|10224738|HY411648   |2015-09-05 13:30:00|043XX S WOOD ST      |0486|BATTERY     |DOMESTIC BATTERY SIMPLE|

In [76]:
rc2.union(one_day).orderBy('Date',ascending=False).show(5)
# we see the next day 3 data is added to the dataframe

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11516594|   JB528186|2018-11-12 00:00:00| 049XX S PRAIRIE AVE|2826|      OTHER OFFENSE|HARASSMENT BY ELE...|               OTHER|

In [77]:
# Now what are the top 10 number of reported crimes by Primary Type in descending order of occurence
rc2.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  46031|
|CRIMINAL SEXUAL A...|    758|
|            STALKING|   3384|
|PUBLIC PEACE VIOL...|  47785|
|           OBSCENITY|    583|
|NON-CRIMINAL (SUB...|      9|
|               ARSON|  11156|
|   DOMESTIC VIOLENCE|      1|
|            GAMBLING|  14422|
|   CRIMINAL TRESPASS| 193371|
|             ASSAULT| 418510|
|      NON - CRIMINAL|     38|
|LIQUOR LAW VIOLATION|  14068|
| MOTOR VEHICLE THEFT| 314131|
|               THEFT|1418446|
|             BATTERY|1232240|
|             ROBBERY| 255598|
|            HOMICIDE|   9466|
|           RITUALISM|     23|
|    PUBLIC INDECENCY|    161|
+--------------------+-------+
only showing top 20 rows



In [78]:
# Now let's sort this result to get top 10
rc2.groupBy('Primary Type').count().orderBy('count',ascending=False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1418446|
|            BATTERY|1232240|
|    CRIMINAL DAMAGE| 771501|
|          NARCOTICS| 711748|
|      OTHER OFFENSE| 418866|
|            ASSAULT| 418510|
|           BURGLARY| 388036|
|MOTOR VEHICLE THEFT| 314131|
| DECEPTIVE PRACTICE| 266293|
|            ROBBERY| 255598|
+-------------------+-------+
only showing top 10 rows



**Challenge Q1**

*Find the percentage of reported crimes that resulted in an arrest*

In [80]:
# Let's first find out if there is any other category for arrest

rc2.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [81]:
# Let's confirm the datatype of Arrest

rc2.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [85]:
# so it's string now lets get the percentage

rc2.filter(col('Arrest') == 'true').count() / rc2.select('Arrest').count()

0.27754481708942097

In [89]:
percentage = rc2.filter(col('Arrest') == 'true').count() / rc2.select('Arrest').count()
print(percentage * 100)

27.754481708942098


**Challenge Q2**

*Find the top 3 locations for reported Crimes*

In [86]:
rc2.groupBy('Location Description').count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770578|
|           RESIDENCE|1145200|
|           APARTMENT| 698460|
+--------------------+-------+
only showing top 3 rows



**Thanks to all!!!**

**This completes session 1 labs for Pyspark**

*Will cover Built in Functions and Rdds in session 2 Labs*

*In case of any query reach out to me below*

[linkedin](https://www.linkedin.com/in/chinmoy07-an)